# keras CNN predict rectangle box

In [1]:
# !pip3 install tensorflow

In [2]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import pickle
from utils_misc import pickle_load_or_create

# Suppres warnings from positioning like
# [h264 @ 0x56bf4fb5da40] reference picture missing during reorder
os.environ['OPENCV_LOG_LEVEL'] = 'OFF'
os.environ['OPENCV_FFMPEG_LOGLEVEL'] = "-8"

2024-08-01 21:32:15.077634: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-01 21:32:15.173839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-01 21:32:16.888795: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [19]:
models = [
    'frames_skillborder_CNN_model_96pixels_history'
]
model_name = models[0]

In [21]:
model = pickle_load_or_create('../models/frames_skillborder_CNN_model_96pixels_history', lambda: None, True)
print(model)

In [22]:
from DataGeneratorFrames import DataGeneratorRectangles

In [24]:
config = pickle_load_or_create(model_name, lambda:{
    'convolution': (3,3),
    'dim':256,
    'rgb':True,
    'unique_labels': 3,
}, config=True)
config

{'convolution': (3, 3), 'dim': 256, 'rgb': True, 'unique_labels': 3}

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, TimeDistributed, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam

if model is None:
    model = Sequential()
    model.add(Conv2D(filters=24, kernel_size=config['convolution'],
                     input_shape=(config['dim'], config['dim'], 3 if config['rgb'] else 1)))
    model.add(MaxPool2D())
    model.add(BatchNormalization())
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3)))
    model.add(MaxPool2D())
    model.add(BatchNormalization())
    
    model.add(Conv2D(filters=48, kernel_size=(3, 3)))
    model.add(MaxPool2D())
    model.add(BatchNormalization())
    
    model.add(Flatten())  # Flatten each frame
    model.add(Dense(64, activation='relu'))
    model.add(Dense(config['unique_labels'], activation='linear'))
    
    model.compile(optimizer=Adam(), 
                  loss='mean_absolute_error', 
                  metrics=['mean_absolute_error', 'mean_squared_error'])
else:
    model = model.model

In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 24)   │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 24)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 127, 127, 24)   │            96 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 32)   │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 48)     │        13,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 30, 30, 48)     │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 43200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,764,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,360,475 (31.89 MB)

 Trainable params: 2,786,755 (10.63 MB)

 Non-trainable params: 208 (832.00 B)

 Optimizer params: 5,573,512 (21.26 MB)

In [27]:
# Parameters
params = {'dim': (config['dim'],config['dim']),
          'n_classes': config['unique_labels'],
          'n_channels': 3 if config['rgb'] else 1,
          'shuffle': True,
}

training_generator = DataGeneratorRectangles(train=True, **params)
test_generator = DataGeneratorRectangles(train=False, **params)

connection established
DataGeneratorSkillBorders init done
on_epoch_end_called
connection established
DataGeneratorSkillBorders init done
on_epoch_end_called


In [28]:
training_generator.batch_order

,videoID,batch_nr_video,frame_start,frame_end,frames,batch_id
0,5,46,1473,1504,32,0
1,11,20,641,672,32,1
2,30,22,705,736,32,2
3,7,20,641,672,32,3
4,75,5,161,192,32,4
...,...,...,...,...,...,...
1012,11,29,929,960,32,1012
1013,52,36,1153,1184,32,1013
1014,152,12,385,416,32,1014
1015,3,23,737,768,32,1015


In [29]:
%%time
X, y = training_generator.__getitem__(25)

 __getitem__(25)
CPU times: user 20.2 ms, sys: 16.1 ms, total: 36.3 ms
Wall time: 43.9 ms


In [30]:
X.shape

(32, 256, 256, 3)

In [31]:
y.shape

(32, 3)

In [32]:
y[:5]

array([[0.610417, 0.665741, 0.9991  ],
       [0.610417, 0.665741, 0.9991  ],
       [0.610417, 0.665741, 0.9991  ],
       [0.610417, 0.665741, 0.9991  ],
       [0.610417, 0.665741, 0.9991  ]])

In [33]:
%%time
history = model.fit(training_generator, epochs=3,
                    validation_data=test_generator, shuffle=False)

 __getitem__(0)
 __getitem__(1)
 __getitem__(0)
 __getitem__(1)
   1/1017 ━━━━━━━━━━━━━━━━━━━━ 1:19:54 5s/step - loss: 0.0221 - mean_absolute_error: 0.0221 - mean_squared_error: 7.0147e-04 __getitem__(2)
   2/1017 ━━━━━━━━━━━━━━━━━━━━ 12:04 714ms/step - loss: 0.0481 - mean_absolute_error: 0.0481 - mean_squared_error: 0.0058    __getitem__(3)
   3/1017 ━━━━━━━━━━━━━━━━━━━━ 12:45 755ms/step - loss: 0.0717 - mean_absolute_error: 0.0717 - mean_squared_error: 0.0149 __getitem__(4)
 __getitem__(5)
   4/1017 ━━━━━━━━━━━━━━━━━━━━ 15:20 909ms/step - loss: 0.0806 - mean_absolute_error: 0.0806 - mean_squared_error: 0.0182 __getitem__(6)
   5/1017 ━━━━━━━━━━━━━━━━━━━━ 14:28 858ms/step - loss: 0.0882 - mean_absolute_error: 0.0882 - mean_squared_error: 0.0214 __getitem__(7)
   6/1017 ━━━━━━━━━━━━━━━━━━━━ 13:59 830ms/step - loss: 0.0938 - mean_absolute_error: 0.0938 - mean_squared_error: 0.0235 __getitem__(8)
   7/1017 ━━━━━━━━━━━━━━━━━━━━ 13:22 795ms/step - loss: 0.0979 - mean_absolute_error: 0.0979

In [34]:
pd.DataFrame(history.history)

,loss,mean_absolute_error,mean_squared_error,val_loss,val_mean_absolute_error,val_mean_squared_error
0,0.125459,0.125459,0.032891,17.867422,17.867422,2242.275391


In [50]:
model.fit?

Signature:
model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose='auto',
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
)
Docstring:
Trains the model for a fixed number of epochs (dataset iterations).

Args:
    x: Input data. It could be:
        - A NumPy array (or array-like), or a list of arrays
        (in case the model has multiple inputs).
        - A tensor, or a list of tensors
        (in case the model has multiple inputs).
        - A dict mapping input names to the corresponding array/tensors,
        if the model has named inputs.
        - A `tf.data.Dataset`. Should return a tuple
        of either `(inputs, targets)` or
        `(inputs, targets, sample_weights)`.
        - A `keras.utils.PyDataset` returning `(inputs,
      

In [36]:
with open(f"../models/{model_name}.pkl", 'wb') as handle:
    pickle.dump(history, handle)